In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

In [2]:
class LayerDense:
    def __init__(self, inputs, neurons):
        self.weights = 0.01 * np.random.randn(inputs, neurons)
        self.biases = np.zeros((1, neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [3]:
class ActivationReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [4]:
class ActivationSoftmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [5]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

In [6]:
class LossCategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
            self.dinputs = -y_true / dvalues
            self.dinputs = self.dinputs / samples

In [7]:
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = ActivationSoftmax()
        self.loss = LossCategoricalCrossEntropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples




In [48]:
X, y = spiral_data(samples=100, classes=3)

dense1 = LayerDense(2, 4)

activation1 = ActivationReLU()
activation2 = ActivationReLU()
activation3 = ActivationReLU()
activation4 = ActivationReLU()
activation5 = ActivationReLU()

dense2 = LayerDense(4, 4)
dense3 = LayerDense(4, 4)
dense4 = LayerDense(4, 4)
dense5 = LayerDense(4, 4)
dense6 = LayerDense(4, 4)

loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

dense1.forward(X)

activation1.forward(dense1.output)

dense2.forward(activation1.output)

activation2.forward(dense2.output)
dense3.forward(activation2.output)

activation3.forward(dense3.output)
dense4.forward(activation3.output)

activation4.forward(dense4.output)
dense5.forward(activation4.output)

activation5.forward(dense5.output)
dense6.forward(activation5.output)

loss = loss_activation.forward(dense6.output, y)

print(loss_activation.output[:5])
print('loss:', loss)

predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

print('acc:', accuracy)

loss_activation.backward(loss_activation.output, y)
dense6.backward(loss_activation.dinputs)
activation5.backward(dense6.dinputs)
dense5.backward(activation5.dinputs)
activation4.backward(dense5.dinputs)
dense4.backward(activation4.dinputs)
activation3.backward(dense4.dinputs)
dense3.backward(activation3.dinputs)
activation2.backward(dense3.dinputs)
dense2.backward(activation2.dinputs)
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

print(dense1.dweights)
print()
print(dense1.dbiases)
print()
print(dense2.dweights)
print()
print(dense2.dbiases)

[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
loss: 1.3862944
acc: 0.3333333333333333
[[ 1.4815319e-11 -4.6593618e-11  7.8607017e-12  7.1904817e-12]
 [ 3.0486512e-12 -3.7357752e-11  2.1892408e-12  1.1766559e-11]]

[[3.6640593e-11 1.7419892e-10 2.7068664e-12 3.6752087e-11]]

[[ 0.0000000e+00  1.2741850e-12  0.0000000e+00  1.1790537e-11]
 [ 0.0000000e+00  1.3330778e-11  6.1919872e-11  1.2428654e-13]
 [ 0.0000000e+00 -2.2159698e-13  1.9022704e-12  9.3252333e-12]
 [ 0.0000000e+00 -3.0859714e-13  0.0000000e+00  1.3609373e-11]]

[[0.0000000e+00 5.8109144e-09 2.6512684e-08 2.7075078e-09]]
